In [33]:
import pandas as pd
import torch

In [34]:
df = pd.read_csv('거문도.csv', encoding='cp949')
df

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,22103,2013-01-01,6.8,1020.6,61.0,6.2,13.9,2.1,1.4,2.1,3.2,5.5,7.1
1,22103,2013-01-02,9.6,1026.8,58.0,4.2,13.8,2.3,1.5,2.0,3.0,4.5,6.4
2,22103,2013-01-03,NaN,1034.6,59.0,-1.6,13.1,NaN,NaN,1.9,2.8,NaN,4.9
3,22103,2013-01-04,5.2,1029.4,47.0,-0.4,11.8,1.5,0.9,1.5,2.4,4.3,5.8
4,22103,2013-01-05,7.3,1024.8,54.0,1.5,11.3,1.4,1.0,1.5,2.2,3.9,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,22103,2022-12-27,8.4,1027.4,60.0,6.0,12.3,1.3,0.9,1.2,1.7,3.6,4.0
3644,22103,2022-12-28,8.8,1029.6,61.0,6.1,12.9,1.3,0.9,1.0,1.6,3.4,3.7
3645,22103,2022-12-29,8.1,1032.0,61.0,5.4,16.5,1.1,0.8,1.1,1.8,3.4,3.7
3646,22103,2022-12-30,7.4,1031.7,55.0,6.5,17.8,1.1,0.7,0.9,1.6,3.3,3.5


In [35]:
df = df.rename(columns={'일시': 'date', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 최대 파고(m)':'wave(m)','평균 풍속(m/s)':'Wind(m/s)','평균기압(hPa)':'Apress(hPa)','평균 상대습도(%)':'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)']] # Column 재정리
df.dropna(subset=['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)'], inplace=True)
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2013-01-01,6.8,1020.6,61.0,6.2,13.9,2.1
2013-01-02,9.6,1026.8,58.0,4.2,13.8,2.3
2013-01-04,5.2,1029.4,47.0,-0.4,11.8,1.5
2013-01-05,7.3,1024.8,54.0,1.5,11.3,1.4
2013-01-06,6.0,1024.1,54.0,2.7,12.3,1.4
...,...,...,...,...,...,...
2022-12-27,8.4,1027.4,60.0,6.0,12.3,1.3
2022-12-28,8.8,1029.6,61.0,6.1,12.9,1.3
2022-12-29,8.1,1032.0,61.0,5.4,16.5,1.1


In [36]:
import pymysql
from sqlalchemy import create_engine, types

# Establish a connection to MySQL
host = 'localhost'
user = 'root'
password = 'k404'
database = 'geomundo'

df = df.reset_index()

conn = pymysql.connect(host=host, user=user, password=password, database=database)

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Convert the DataFrame to a SQL table with specified column types
table_name = 'weather_data'
dtype = {'date': types.Date(), 'Wind (m/s)': types.Float(), 'Apress (hPa)': types.Float(), 'Moisture (%)': types.Float(), 'Atemp (°C)': types.Float(), 'Stemp (°C)': types.Float(), 'wave (m)': types.Float()}
df.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype)

# Close the connection
conn.close()


In [37]:
df

,date,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
0,2013-01-01,6.8,1020.6,61.0,6.2,13.9,2.1
1,2013-01-02,9.6,1026.8,58.0,4.2,13.8,2.3
2,2013-01-04,5.2,1029.4,47.0,-0.4,11.8,1.5
3,2013-01-05,7.3,1024.8,54.0,1.5,11.3,1.4
4,2013-01-06,6.0,1024.1,54.0,2.7,12.3,1.4
...,...,...,...,...,...,...,...
3461,2022-12-27,8.4,1027.4,60.0,6.0,12.3,1.3
3462,2022-12-28,8.8,1029.6,61.0,6.1,12.9,1.3
3463,2022-12-29,8.1,1032.0,61.0,5.4,16.5,1.1
3464,2022-12-30,7.4,1031.7,55.0,6.5,17.8,1.1
